# PySpark

In [14]:
# find the env spark
import findspark
findspark.init()

import pyspark

 # khởi tạo trình chạy pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lesson1').getOrCreate()
spark 

In [2]:
# check cores
spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

1

## DataFrame Creation

In [13]:
# create from row
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [7]:
# create from tuple
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [8]:
# Create a PySpark DataFrame from an RDD consisting of a list of tuples.
rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [23]:
# from pandas.dataframe
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

## Viewing

In [21]:
df.limit(3).show()

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
|  1|         4.9|        3.0|         1.4|        0.2| setosa|
|  2|         4.7|        3.2|         1.3|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+



In [10]:
# show sample
df.show(1)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 1 row



In [15]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [28]:
# len df 
df.count()

3

In [14]:
spark.sql.repl.eagerEval.maxNumRows # setup maxrow to show
spark.conf.set('spark.sql.repl.eagerEval.enabled', True) # setup config to show in jupyter
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [11]:
# dtype
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [12]:
# convert pandas.df
df.toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [16]:
# show columns
df.columns

['a', 'b', 'c', 'd', 'e']

## Reading file

In [3]:
import os
file = os.path.normpath(os.path.join(os.getcwd(),"..","0. Data",'datatest.csv')) ## ".." to get out folder
file

'J:\\My Drive\\GitCode\\My_learning\\11. BigData\\0. Data\\datatest.csv'

In [4]:
df = spark.read.csv(file,header=True)
df.show(1)

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 1 row



In [23]:
# csv
import os
# file = os.path.normpath(os.path.join(os.getcwd(),"..","2.Dask",'datatest','1989.csv')) ## ".." to get out folder
df = spark.read.csv(r"J:\My Drive\GitCode\My_learning\11. BigData\0. Data\datatest.csv",header=True)
df.show(1)

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 1 row



In [19]:
import myfunction as mf
mf.display(df.limit(4).toPandas())

,_c0,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa


In [ ]:
# read parquet
df = spark.read.parquet('data.parquet') # read file parquet

# read many file parquet with the sample form
df_all = spark.read.parquet('data*.parquet')
df1_2 = spark.read.parquet('data1.parquet','data2.parquet')
df = spark.read.option('bathPath',path).parquet(path + '//data*.parquet') # set option
df = spark.read.parquet(folder1 +"//data*.parquet", folder2 +"//*") # set option

## Validation

- set schema to datatype

In [22]:
df.printSchema() # get dtype by SAMPLE of beginning datafile

root
 |-- _c0: string (nullable = true)
 |-- sepal_length: string (nullable = true)
 |-- sepal_width: string (nullable = true)
 |-- petal_length: string (nullable = true)
 |-- petal_width: string (nullable = true)
 |-- species: string (nullable = true)



In [23]:
df.describe()

DataFrame[summary: string, _c0: string, sepal_length: string, sepal_width: string, petal_length: string, petal_width: string, species: string]

In [25]:
df.schema['sepal_length']

StructField('sepal_length', StringType(), True)

In [82]:
from pyspark.sql.types import *

# set schema
data_schema = [
            # StructField('_c0',IntegerType()),
              StructField('sepal_length',FloatType()),
              StructField('sepal_width',FloatType()),
              # StructField('petal_length',FloatType()), # dont need to set all
              StructField('petal_width',FloatType()),
              StructField('species',StringType()),
              ]

final_struc = StructType(fields = data_schema)

df = spark.read.csv(file,header=True,schema = final_struc)

df.printSchema() # chỉ đọc những cột có định nghĩa trong final_struc schema

root
 |-- sepal_length: float (nullable = true)
 |-- sepal_width: float (nullable = true)
 |-- petal_width: float (nullable = true)
 |-- species: string (nullable = true)



In [26]:
df.show(2)
df.printSchema()

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
|  1|         4.9|        3.0|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 2 rows

root
 |-- _c0: string (nullable = true)
 |-- sepal_length: string (nullable = true)
 |-- sepal_width: string (nullable = true)
 |-- petal_length: string (nullable = true)
 |-- petal_width: string (nullable = true)
 |-- species: string (nullable = true)



In [ ]:
# change the dtype columns
from pyspark.sql.types import *

df = df.withColumn('sepal_length',df['sepal_length'].cast(FloatType()) )\
        .withColumn('sepal_width',df['sepal_width'].cast(IntegerType()) )\
        .withColumn('sepal_length',to_date(df['sepal_length'],'yy.dd.mm') )\
        .withColumn('sepal_length',to_timestamp(df['petal_width'],'yy.dd.mm'))

## Write data

In [3]:
df = spark.read.csv(r"J:\My Drive\GitCode\My_learning\11. BigData\0. Data\datatest.csv",header=True)

In [ ]:
df.write.csv("/Volumes/GoogleDrive-106231888590528523181/My Drive/GitCode/My_learning/11. BigData/3. Spark/ab",'overwrite')

In [21]:
from py4j.java_gateway import java_import

In [22]:
java_import(spark._jvm,"org.apache.hadoop.fs.Path")

In [ ]:
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
file = fs.globStatus(spark._jvm.Path('write_test.csv/part*'))[0].getPath().getName()

In [ ]:
df.write.parquet("namesAndFavColors.parquet")

## Selecting and accessing data

In [26]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

In [17]:
# select specific columns
df.select("a", "b", "c")#.describe().show()

a,b,c
1,2.0,string1
2,3.0,string2
3,4.0,string3


In [29]:
df.select("*")

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [26]:
df.select('sepal_length','sepal_width','petal_length').summary('count','max','min').show()

+-------+------------+-----------+------------+
|summary|sepal_length|sepal_width|petal_length|
+-------+------------+-----------+------------+
|  count|         150|        150|         150|
|    max|         7.9|        4.4|         6.9|
|    min|         4.3|        2.0|         1.0|
+-------+------------+-----------+------------+



In [19]:
df.select(df.c.isNull()).show()

+-----------+
|(c IS NULL)|
+-----------+
|      false|
|      false|
|      false|
+-----------+



In [27]:
# add new column
df.withColumn('upper_c', upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|upper_c|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|STRING2|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



In [23]:
#filter
df.filter(df.a == 1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [3]:
# csv
import os
# file = os.path.normpath(os.path.join(os.getcwd(),"..","2.Dask",'datatest','1989.csv')) ## ".." to get out folder
df = spark.read.csv(r"J:\My Drive\GitCode\My_learning\11. BigData\0. Data\datatest.csv",header=True)
df.show(1)

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 1 row



In [5]:
# rename columns
df.withColumnRenamed('sepal_length','col1').show(2)

+---+----+-----------+------------+-----------+-------+
|_c0|col1|sepal_width|petal_length|petal_width|species|
+---+----+-----------+------------+-----------+-------+
|  0| 5.1|        3.5|         1.4|        0.2| setosa|
|  1| 4.9|        3.0|         1.4|        0.2| setosa|
+---+----+-----------+------------+-----------+-------+
only showing top 2 rows



In [10]:
# truncate to show all value in cell, not "..."
df.select(['sepal_length','sepal_width','petal_length']).show(3,truncate = False) 

+------------+-----------+------------+
|sepal_length|sepal_width|petal_length|
+------------+-----------+------------+
|5.1         |3.5        |1.4         |
|4.9         |3.0        |1.4         |
|4.7         |3.2        |1.3         |
+------------+-----------+------------+
only showing top 3 rows



In [12]:
# sort value
df.select(['sepal_length','sepal_width','petal_length']).orderBy(df['sepal_length']).show(5)
df.select(['sepal_length','sepal_width','petal_length']).orderBy(df['sepal_length'].desc()).show(5) # sort desc 
df.orderBy(df['sepal_length'].desc(),df['sepal_width']).show() # multi columns to sort

+------------+-----------+------------+
|sepal_length|sepal_width|petal_length|
+------------+-----------+------------+
|         4.3|        3.0|         1.1|
|         4.4|        3.2|         1.3|
|         4.4|        3.0|         1.3|
|         4.4|        2.9|         1.4|
|         4.5|        2.3|         1.3|
+------------+-----------+------------+
only showing top 5 rows



## Filter

In [15]:
import os
# file = os.path.normpath(os.path.join(os.getcwd(),"..","2.Dask",'datatest','1989.csv')) ## ".." to get out folder
df = spark.read.csv(r"J:\My Drive\GitCode\My_learning\11. BigData\0. Data\datatest.csv",header=True)
df.show(1)

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 1 row



In [16]:
#filter
df.filter(df.sepal_length == 5.1).show()

+---+------------+-----------+------------+-----------+----------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|   species|
+---+------------+-----------+------------+-----------+----------+
|  0|         5.1|        3.5|         1.4|        0.2|    setosa|
| 17|         5.1|        3.5|         1.4|        0.3|    setosa|
| 19|         5.1|        3.8|         1.5|        0.3|    setosa|
| 21|         5.1|        3.7|         1.5|        0.4|    setosa|
| 23|         5.1|        3.3|         1.7|        0.5|    setosa|
| 39|         5.1|        3.4|         1.5|        0.2|    setosa|
| 44|         5.1|        3.8|         1.9|        0.4|    setosa|
| 46|         5.1|        3.8|         1.6|        0.2|    setosa|
| 98|         5.1|        2.5|         3.0|        1.1|versicolor|
+---+------------+-----------+------------+-----------+----------+



In [21]:
# where va filter la nhu nhau 
df.select(['sepal_length','sepal_width','petal_length','species']).where(df.species.like("%eto%")).show(2)

+------------+-----------+------------+-------+
|sepal_length|sepal_width|petal_length|species|
+------------+-----------+------------+-------+
|         5.1|        3.5|         1.4| setosa|
|         4.9|        3.0|         1.4| setosa|
+------------+-----------+------------+-------+
only showing top 2 rows



In [23]:
df.select('species',df.species.substr(-4,3)).show(2) # string lấy substring từ -4, 3 ký tu

+-------+-------------------------+
|species|substring(species, -4, 3)|
+-------+-------------------------+
| setosa|                      tos|
| setosa|                      tos|
| setosa|                      tos|
+-------+-------------------------+
only showing top 3 rows



In [27]:
df[df.species.isin('setosa','versicolor')].show(2) # isin
df[df.species.startswith('s')].show(2) # startswith

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
|  1|         4.9|        3.0|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 2 rows

+---+------------+-----------+------------+-----------+-------+
|_c0|sepal_length|sepal_width|petal_length|petal_width|species|
+---+------------+-----------+------------+-----------+-------+
|  0|         5.1|        3.5|         1.4|        0.2| setosa|
|  1|         4.9|        3.0|         1.4|        0.2| setosa|
+---+------------+-----------+------------+-----------+-------+
only showing top 2 rows



In [30]:
df.select(df.columns[:3]).show(2)

+---+------------+-----------+
|_c0|sepal_length|sepal_width|
+---+------------+-----------+
|  0|         5.1|        3.5|
|  1|         4.9|        3.0|
+---+------------+-----------+
only showing top 2 rows



In [64]:
# collect
a = df.where('sepal_length > 6').collect()
print(type(a[0]))
print(a[1][-1])

<class 'pyspark.sql.types.Row'>
virginica


In [40]:
# slice
from pyspark.sql.functions import collect_list, slice
df2 = df.groupBy('species').agg(collect_list("sepal_length").alias("sepal_length_list"))
df2.show()
df2.select(slice(df2.sepal_length_list,2,2).alias('col1')).show() # slice(self, start, leght)

+----------+--------------------+
|   species|   sepal_length_list|
+----------+--------------------+
| virginica|[6.3, 5.8, 7.1, 6...|
|versicolor|[7.0, 6.4, 6.9, 5...|
|    setosa|[5.1, 4.9, 4.7, 4...|
+----------+--------------------+

+----------+
|      col1|
+----------+
|[5.8, 7.1]|
|[6.4, 6.9]|
|[4.9, 4.7]|
+----------+



## Apply function

In [24]:
# use the APIs in a pandas Series within Python native function

import pandas
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select("*",pandas_plus_one('a').alias('age')).show() # select * rename columns as 'age'

+---+---+-------+----------+-------------------+---+
|  a|  b|      c|         d|                  e|age|
+---+---+-------+----------+-------------------+---+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|  2|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|  3|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|  4|
+---+---+-------+----------+-------------------+---+



In [22]:
from pyspark.sql.functions import expr
df.show(3)
df.withColumn('percent',expr("b/10*100")).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|percent|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|   20.0|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|   30.0|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|   50.0|
+---+---+-------+----------+-------------------+-------+



## Grouping Data

In [9]:
from pyspark.sql.functions import col as scol, sum as ssum, avg as savg, max as smax, count as scount

In [38]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [11]:
# groupby color with mean
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [12]:
df.groupBy("color","fruit").sum("v1","v2").show()

+-----+------+-------+-------+
|color| fruit|sum(v1)|sum(v2)|
+-----+------+-------+-------+
|  red|banana|      8|     80|
| blue|banana|      2|     20|
|  red|carrot|      8|     80|
| blue| grape|      4|     40|
|black|carrot|      6|     60|
|  red| grape|      8|     80|
+-----+------+-------+-------+



In [13]:
df.groupBy("color").agg(savg("v1").alias("avg_v1"), ssum("v2").alias("sum_v2"), smax("v2").alias("max_v2") ).show()

+-----+------+------+------+
|color|avg_v1|sum_v2|max_v2|
+-----+------+------+------+
|  red|   4.8|   240|    80|
| blue|   3.0|    60|    40|
|black|   6.0|    60|    60|
+-----+------+------+------+



In [14]:
# tuong tu having sql
df.groupBy("color")\
.agg(savg("v1").alias("avg_v1"), 
     ssum("v2").alias("sum_v2"), 
     smax("v2").alias("max_v2") )\
.where(scol("max_v2") >= 50) \
.show(truncate=False)

+-----+------+------+------+
|color|avg_v1|sum_v2|max_v2|
+-----+------+------+------+
|red  |4.8   |240   |80    |
|black|6.0   |60    |60    |
+-----+------+------+------+



In [15]:
# custom function for groupby
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

@F.udf(returnType=FloatType()) # define return float
def custom_func(x):
    return x[0] - x[-1]

df.groupby('color')\
.agg(F.collect_list("v1").alias("v1_groupby_color"))\
.withColumn('v1_custom', custom_func('v1_groupby_color'))\
.show()

+-----+----------------+---------+
|color|v1_groupby_color|v1_custom|
+-----+----------------+---------+
|  red| [1, 3, 5, 7, 8]|     null|
| blue|          [2, 4]|     null|
|black|             [6]|     null|
+-----+----------------+---------+



In [42]:
# udf
from pyspark.sql.functions import udf

@udf(returnType = FloatType())
def square_float(x):
    return float(x**2)

df.select('v1',square_float('v1').alias('v1_squared')).show()

+---+----------+
| v1|v1_squared|
+---+----------+
|  1|       1.0|
|  2|       4.0|
|  3|       9.0|
|  4|      16.0|
|  5|      25.0|
|  6|      36.0|
|  7|      49.0|
|  8|      64.0|
+---+----------+



In [19]:
# multi columns in groupby
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType
import numpy as np
@pandas_udf(DoubleType(), functionType=PandasUDFType.GROUPED_AGG)
def f(x, y):
    return np.mean([np.min(x), np.min(y)])

df.groupBy("color").agg(f("v1", "v2").alias("avg_min")).show()

+-----+-------+
|color|avg_min|
+-----+-------+
|black|   33.0|
| blue|   11.0|
|  red|    5.5|
+-----+-------+



In [20]:
@pandas_udf("color string, fruit string, v1 bigint, v2 bigint, v1_ float",functionType = PandasUDFType.GROUPED_MAP)  
def normalize(df):
    v1 = df.v1
    return df.assign(v1_=(v1 - v1.mean()) / v1.std())

df.groupby("color").apply(normalize).show()  

C:\Users\User\anaconda3\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-----+------+---+---+-----------+
|color| fruit| v1| v2|        v1_|
+-----+------+---+---+-----------+
|black|carrot|  6| 60|       null|
| blue|banana|  2| 20|-0.70710677|
| blue| grape|  4| 40| 0.70710677|
|  red|banana|  1| 10| -1.3270175|
|  red|carrot|  3| 30|-0.62858725|
|  red|carrot|  5| 50| 0.06984303|
|  red|banana|  7| 70| 0.76827335|
|  red| grape|  8| 80|  1.1174885|
+-----+------+---+---+-----------+



## Spark SQL

In [18]:
# set df table name
df.createOrReplaceTempView('table1')
spark.sql('select * from table1').limit(10).toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [ ]:
from pyspark.sql.types import sqlContext
sqlContext.registerDataFrameAsTable(df, "df_table")

median_rating = sqlContext.sql("""
    SELECT percentile(v2, 0.5) AS median_rating 
    FROM df_table
""").first()["median_rating"]

print("Median rating:", median_rating)

In [19]:
from pyspark.ml.feature import SQLTransformer
sqlTrans = SQLTransformer(statement='select * from __THIS__')
sqlTrans.transform(df).show(2)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 2 rows



In [ ]:
# from pyspark.conf import SparkConf
# conf = SparkConf()  # create the configuration
# conf.set("spark.jars", r"C:\Spark_env\ojdbc11.jar")  # set the spark.jars
driver ="oracle.jdbc.driver.OracleDriver"# 'oracle.jdbc.driver.OracleDriver'
url = 'jdbc:oracle:thin:@192.168.18.32:1521/DTT'
user = 'datkt'
password = 'hct5Kg'
table = 'DTT_SD.IRIS'

df = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .load()